In [62]:
# Import the dependencies
import pandas as pd
import numpy as np
import datetime

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns

# ORM Stuff
from sqlalchemy import create_engine, inspect, text, func


In [63]:
 # Create engine using the `Olympic_Swimming_Results.sqlite` database file
engine = create_engine("sqlite:///Olympic_Swimming_Results.sqlite")

In [64]:
 # Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Using the inspector to print the column names within the 'dow' table and its types
for table in tables:
    print(table)
    print("--------")
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])

    print()

Olympic_Swimming_Results
--------
Location TEXT
Year BIGINT
Distance (in meters) TEXT
Stroke TEXT
Gender TEXT
Team TEXT
Athlete TEXT
Results TEXT
Rank BIGINT



In [65]:
conn = engine.connect() # Raw SQL/Pandas

In [66]:
 # Raw SQL
query = text("""SELECT COUNT(*) AS total_records FROM Olympic_Swimming_Results;""")
df1 = pd.read_sql(query, con=conn)
df1.head(10)

,total_records
0,42269


Question 1: What Country had the fastest Swimmers?

In [70]:
# Raw SQL
query1 = text("""
    SELECT 
        year, 
        AVG(CAST("Distance (in meters)" AS FLOAT)) AS avg_distance_in_meters,
        COUNT(DISTINCT Stroke) AS unique_strokes,
        COUNT(DISTINCT Gender) AS genders,
        COUNT(DISTINCT Team) AS unique_teams,
        COUNT(DISTINCT Athlete) AS unique_athletes,
        AVG(CAST(Results AS FLOAT)) AS avg_results,
        AVG(Rank) AS avg_rank
    FROM 
        Olympic_Swimming_Results
    GROUP BY 
        year
    ORDER BY 
        year;
""")
df2 = pd.read_sql(query1, con=conn)
print(df2.shape)
df2.head(10)

(25, 8)


,Year,avg_distance_in_meters,unique_strokes,genders,unique_teams,unique_athletes,avg_results,avg_rank
0,1912,361.293333,3,2,14,50,0.000000,1.631667
1,1920,413.578512,3,2,10,38,1.995868,2.349174
2,1924,305.340454,3,2,19,68,0.787717,2.097463
3,1928,289.408039,3,2,23,72,1.442144,2.394641
4,1932,269.338776,3,2,15,68,4.649252,2.677551
5,1936,283.064516,3,2,18,79,4.731682,2.872120
6,1948,281.992933,3,2,16,78,4.606832,3.187279
7,1952,280.649652,3,2,19,82,5.410673,3.146172
8,1956,264.807805,4,2,20,95,4.564585,3.187317
9,1960,231.144568,5,2,19,110,3.827032,3.194183


Stacked bar chart

In [ ]:
# Set year as index for stacking
df2.set_index("year", inplace=True)


# Define categories for stacking
categories = [
    "unique_strokes",
    "unique_teams",
    "unique_athletes",
    "avg_results",
    "avg_rank",
]

# Define category labels for visualization
category_labels = [
    "Types of Strokes",
    "Teams",
    "Athletes",  
    "Results",
    "Rank",
]

# Define colors for each category
colors = ["blue", "pink", "purple", "red", "gray", "orange"]

# Initialize bottom values for stacking
bottom_values = np.zeros(len(df2))

# Create figure and axis
fig, ax = plt.subplots(figsize=(12, 6))

# Loop through categories to create stacked bars
for idx, category in enumerate(categories):
    ax.bar(df2.index, df2[category], bottom=bottom_values, 
           label=category_labels[idx], color=colors[idx])
    bottom_values += df2[category].values  

# Labels and title
ax.set_xlabel("Year")
ax.set_ylabel("Average Value")  
ax.set_title("Swimming Performance Trends Over Time (Stacked Bar Chart)")
ax.set_xticks(df2.index)  # 
ax.set_xticklabels(df2.index, rotation=45)  
ax.legend(title="Performance Metrics") 
ax.grid(axis="y", linestyle="--", alpha=0.5)

# Show plot
plt.show()
